In [4]:
! pip install fuzzywuzzy
! pip install python-Levenshtein

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
df=pd.read_excel("master.xlsx")
df['cc'] = df['cc'].astype(str)
df['seating_capacity'] = df['seating_capacity'].astype(str)
df.head()

,make,model,variant,fuel_type,cc,seating_capacity,output
0,ALFA ROMEO,156 GTA,STD,Petrol,3179,5,1110110101
1,ALFA ROMEO,VINTAGE,ROMEO,Petrol,1100,5,1110110201
2,ALFA ROMEO,VINTAGE,ALFA ROMEO,Petrol,1750,5,1110110202
3,ALFA ROMEO,VINTAGE,ALFA ROMEO,Petrol,1570,5,1110110203
4,ALFA ROMEO,VINTAGE,ALFA ROMEO,Petrol,1110,5,1110110204


In [6]:
#df.drop(df.iloc[:, 7:26], inplace=True, axis=1)
df.head()

,make,model,variant,fuel_type,cc,seating_capacity,output
0,ALFA ROMEO,156 GTA,STD,Petrol,3179,5,1110110101
1,ALFA ROMEO,VINTAGE,ROMEO,Petrol,1100,5,1110110201
2,ALFA ROMEO,VINTAGE,ALFA ROMEO,Petrol,1750,5,1110110202
3,ALFA ROMEO,VINTAGE,ALFA ROMEO,Petrol,1570,5,1110110203
4,ALFA ROMEO,VINTAGE,ALFA ROMEO,Petrol,1110,5,1110110204


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7678 entries, 0 to 7677
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   make              7678 non-null   object
 1   model             7678 non-null   object
 2   variant           7678 non-null   object
 3   fuel_type         7678 non-null   object
 4   cc                7678 non-null   object
 5   seating_capacity  7678 non-null   object
 6   output            7678 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 420.0+ KB


In [9]:

inputs=['make','model','variant','cc','seating_capacity','fuel_type']
x = df[inputs] # Features
y = df.output # Target variable
x

,make,model,variant,cc,seating_capacity,fuel_type
0,ALFA ROMEO,156 GTA,STD,3179,5,Petrol
1,ALFA ROMEO,VINTAGE,ROMEO,1100,5,Petrol
2,ALFA ROMEO,VINTAGE,ALFA ROMEO,1750,5,Petrol
3,ALFA ROMEO,VINTAGE,ALFA ROMEO,1570,5,Petrol
4,ALFA ROMEO,VINTAGE,ALFA ROMEO,1110,5,Petrol
...,...,...,...,...,...,...
7673,THE LONDON TAXI COMPANY,TX4,LONDON TAXI,2499,5,Diesel
7674,HISPANO SUIZA,VINTAGE,VINTAGE,6597,5,Diesel
7675,DETROITER,VINTAGE,TYPE A SPEEDSTER,3000,2,Petrol
7676,HUPMOBILE,VINTAGE,MODEL 20 RUNABOUT,1800,2,Petrol


In [10]:
df_train=pd.read_excel("train.xlsx")
df_train['cc'] = df_train['cc'].astype(str)
df_train['seating_capacity'] = df_train['seating_capacity'].astype(str)
df_test=pd.read_csv("test_mine.csv")

In [11]:
df_train

,make,model,variant,cc,seating_capacity,fuel_type,local_output
0,HONDA,CR-V,2.4 MT,2354,5,PETROL,12623
1,HONDA,CR-V,2.4 MT,2354,5,PETROL,12623
2,VOLKSWAGEN,AMEO,1.5L TDI MT COMFORTLINE PLUS,1498,5,DIESEL,36811
3,FORD,FIGO,1.2 TITANIUM SPORTS PETROL,1196,5,PETROL,36819
4,LAMBORGHINI,AVENTADOR,S LP 740-4,6498,2,PETROL,37548
...,...,...,...,...,...,...,...
5732,TATA MOTORS LTD,TIGOR,XZ CNG,1199,5,CNG,46395
5733,TATA MOTORS LTD,TIGOR,REVOTRON XZ(O),1199,5,PETROL,36226
5734,TATA MOTORS LTD,TIGOR,REVOTRON XT,1199,5,PETROL,36224
5735,FORD,IKON,1.3 IRIS,1299,5,PETROL,46464


In [12]:
df_train.drop(df_train.iloc[:, 7:26], inplace=True, axis=1)

In [13]:
df_train

,make,model,variant,cc,seating_capacity,fuel_type,local_output
0,HONDA,CR-V,2.4 MT,2354,5,PETROL,12623
1,HONDA,CR-V,2.4 MT,2354,5,PETROL,12623
2,VOLKSWAGEN,AMEO,1.5L TDI MT COMFORTLINE PLUS,1498,5,DIESEL,36811
3,FORD,FIGO,1.2 TITANIUM SPORTS PETROL,1196,5,PETROL,36819
4,LAMBORGHINI,AVENTADOR,S LP 740-4,6498,2,PETROL,37548
...,...,...,...,...,...,...,...
5732,TATA MOTORS LTD,TIGOR,XZ CNG,1199,5,CNG,46395
5733,TATA MOTORS LTD,TIGOR,REVOTRON XZ(O),1199,5,PETROL,36226
5734,TATA MOTORS LTD,TIGOR,REVOTRON XT,1199,5,PETROL,36224
5735,FORD,IKON,1.3 IRIS,1299,5,PETROL,46464


In [14]:
df_train=df_train.drop_duplicates()

In [15]:
#df_train['local_output'] = df_train['local_output'].astype("int64")
# Setting the index of the dataframe to the Date column so that time series plots could be shown
#df_train = df_train.set_index('local_output')
df_train
print(df_train['local_output'][2])


36811


In [21]:
simi=0
max1=0
output=[]
index1=0
df_train.set_index("output", inplace = True)
for i in df_test.index:
    max1=0
    l=[]
    lo_ou=df_test['output'][i]
    result=df_train.loc[lo_ou]
    for _ in range(6):
        l.append(result[_])
    for j in df.index:
        train_str="".join(l)
        list_master=[df['make'][j],df['model'][j],df['variant'][j],df['cc'][j],df['seating_capacity'][j],df['fuel_type'][j]]
        master_str=" ".join(list_master)
        #print(train_str)
        #print(master_str)
        simi=fuzz.token_set_ratio(master_str, train_str)
        if(simi>max1):
            max1=simi
            index1=j
            #print(index1)
    #print(output)
    #print(df['output'][index1])
    output.append(df['output'][index1])
    simi=0


KeyError: "None of ['output'] are in the columns"

In [22]:
output

[]

In [23]:
df_test['mapped_code']=output

ValueError: Length of values (0) does not match length of index (9)

In [24]:
df_test

,12623
0,12623
1,36811
2,36819
3,37548
4,37128
5,36998
6,37726
7,12792
8,33273


In [ ]:
#df_test.to_csv('./submission.csv')